In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_data_S9
from  Utils import test,train,plot_acc_loss
from models import model_summary

device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu')
print(device)

c:\Users\Aai\anaconda3\envs\pytorch\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


cuda


In [2]:
train_loader , test_loader = load_data_S9()

CUDA Available? True
Files already downloaded and verified
Files already downloaded and verified


In [3]:
class S9_model(nn.Module):
    def __init__(self):
        super(S9_model,self).__init__()


        self.convblock1 = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=128,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Dropout2d(0.1),
            nn.Conv2d(in_channels=128,out_channels=32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.Dropout2d(0.1),
            nn.BatchNorm2d(32),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=1,groups=32),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(0.1),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=2,dilation=2),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=2,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(0.1)
        )
        self.convblock2 = nn.Sequential(
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.1),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.1),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=1,groups=32),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(0.1),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=2,dilation=4),
            nn.Conv2d(in_channels=32,out_channels=16,kernel_size=3,stride=2,padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout2d(0.1)
        )
        self.convblock3 = nn.Sequential(
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.1),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Dropout2d(0.1),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=1,groups=32),
            nn.Conv2d(in_channels=32,out_channels=64,kernel_size=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Dropout2d(0.1),
            nn.Conv2d(in_channels=64,out_channels=32,kernel_size=3,padding=2,dilation=2),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=2,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(0.1)
        )
        self.convblock4 = nn.Sequential(
            nn.Conv2d(in_channels=32,out_channels=16,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),
            nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.1),
            nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3,padding=1,groups=16),
            nn.Conv2d(in_channels=16,out_channels=16,kernel_size=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Dropout2d(0.1),
            nn.Conv2d(in_channels=16,out_channels=16,kernel_size=3,padding=2,dilation=2),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Dropout2d(0.1)
        )
        self.gap = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(in_features=32,out_features=10)


    def forward(self,x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.gap(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        x = F.log_softmax(x,dim=1)
        return x

In [4]:
model = S9_model().to(device)
model_summary(model,input_size=(3,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 32, 32]           3,584
              ReLU-2          [-1, 128, 32, 32]               0
       BatchNorm2d-3          [-1, 128, 32, 32]             256
         Dropout2d-4          [-1, 128, 32, 32]               0
            Conv2d-5           [-1, 32, 32, 32]          36,896
              ReLU-6           [-1, 32, 32, 32]               0
         Dropout2d-7           [-1, 32, 32, 32]               0
       BatchNorm2d-8           [-1, 32, 32, 32]              64
            Conv2d-9           [-1, 32, 32, 32]             320
           Conv2d-10           [-1, 32, 32, 32]           1,056
      BatchNorm2d-11           [-1, 32, 32, 32]              64
             ReLU-12           [-1, 32, 32, 32]               0
        Dropout2d-13           [-1, 32, 32, 32]               0
           Conv2d-14           [-1, 32,

In [5]:

criterion = F.nll_loss
num_epochs = 30
optimizer_sgd = optim.SGD(model.parameters(), lr=0.01,momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer_sgd,gamma=0.99,step_size=5)


for epoch in range(1, num_epochs+1):
  print(f'Epoch {epoch}')
  train(model, device, train_loader, optimizer_sgd,criterion)
  scheduler.step()
  test(model, device, test_loader, criterion)


Epoch 1


100%|██████████| 20/20 [00:09<00:00,  2.11it/s]


Test set: Average loss: 0.0041, Accuracy: 2214/10000 (22.14%)

Epoch 2


100%|██████████| 20/20 [00:09<00:00,  2.13it/s]


Test set: Average loss: 0.0037, Accuracy: 3209/10000 (32.09%)

Epoch 3


...

Test set: Average loss: 0.0022, Accuracy: 6015/10000 (60.15%)

Epoch 29


100%|██████████| 20/20 [00:09<00:00,  2.07it/s]


Test set: Average loss: 0.0022, Accuracy: 5998/10000 (59.98%)

Epoch 30


100%|██████████| 20/20 [00:09<00:00,  2.07it/s]

Test set: Average loss: 0.0022, Accuracy: 6020/10000 (60.20%)

